In [ ]:
# pip install dask-geopandas

In [9]:
# Import libraries
import dask.dataframe as dd
import dask_geopandas
import geopandas as gpd

In [10]:
# Read dataset CSV
ddf = dd.read_csv(r"Google_BU\Rentachintala.csv")
ddf.head()

,latitude,longitude,area_in_meters,confidence,geometry,full_plus_code
0,16.722812,80.530759,16.5662,0.7102,"POLYGON((80.5307812670313 16.7227951068821, 80...",7M82PGFJ+48CF
1,20.832199,81.723316,93.8762,0.7002,"POLYGON((81.723335342909 20.8321350986296, 81....",7MG3RPJF+V8GR
2,21.243695,80.274293,50.7108,0.7765,"POLYGON((80.2743130059794 21.2436510721232, 80...",7MH267VF+FPFM
3,20.052024,82.588784,24.7667,0.7542,"POLYGON((82.588803295914 20.051995753738, 82.5...",7MG43H2Q+RG3R
4,22.091796,81.711124,24.2144,0.7995,"POLYGON((81.711138635493 22.0917657080355, 81....",7MJ33PR6+PC9X


In [11]:
gddf = dask_geopandas.from_dask_dataframe(ddf,
                                          geometry=ddf['geometry']
                                          .map_partitions(gpd.GeoSeries.from_wkt,
                                                          meta=gpd.GeoSeries([])))

gddf = gddf.reset_index()
gddf.head()

,index,latitude,longitude,area_in_meters,confidence,geometry,full_plus_code
0,0,16.722812,80.530759,16.5662,0.7102,"POLYGON ((80.53078 16.72280, 80.53078 16.72282...",7M82PGFJ+48CF
1,1,20.832199,81.723316,93.8762,0.7002,"POLYGON ((81.72334 20.83214, 81.72337 20.83224...",7MG3RPJF+V8GR
2,2,21.243695,80.274293,50.7108,0.7765,"POLYGON ((80.27431 21.24365, 80.27433 21.24373...",7MH267VF+FPFM
3,3,20.052024,82.588784,24.7667,0.7542,"POLYGON ((82.58880 20.05200, 82.58880 20.05205...",7MG43H2Q+RG3R
4,4,22.091796,81.711124,24.2144,0.7995,"POLYGON ((81.71114 22.09177, 81.71115 22.09182...",7MJ33PR6+PC9X


In [12]:
gddf.crs = 'EPSG:4326'

In [13]:
aoi = gpd.read_file(r"Reprojected\Rentachintala.shp")
aoi.head()

,District,STATE,TEHSIL,Shape_Leng,Shape_Area,geometry
0,GUNTUR,ANDHRA PRADESH,RENTACHINTALA,72665.440465,1.941671e+08,"POLYGON ((79.52718 16.62260, 79.52497 16.61985..."


In [14]:
clipped = dask_geopandas.clip(gddf, aoi)
clipped.head()

,index,latitude,longitude,area_in_meters,confidence,geometry,full_plus_code
197859,197859,16.487608,79.539102,71.1922,0.8666,"POLYGON ((79.53905 16.48758, 79.53906 16.48764...",7J8XFGQQ+2JX7
137914,137914,16.495057,79.529501,55.0820,0.7311,"POLYGON ((79.52955 16.49502, 79.52944 16.49505...",7J8XFGWH+2RC6
101619,101619,16.519088,79.503959,61.0406,0.7040,"POLYGON ((79.50398 16.51904, 79.50391 16.51906...",7J8XGG93+JHPG
84332,84332,16.520258,79.505311,55.6385,0.8318,"POLYGON ((79.50533 16.52021, 79.50528 16.52022...",7J8XGGC4+4439
134938,134938,16.520668,79.504216,114.2039,0.7903,"POLYGON ((79.50424 16.52060, 79.50417 16.52061...",7J8XGGC3+7M8Q


In [15]:
clipped = clipped.compute()
clipped.to_file(r"Clipped\Rentachintala_clipped.shp")

C:\Users\AbhishekRawat\AppData\Local\Temp\ipykernel_25716\1757001551.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  clipped.to_file(r"Clipped\Rentachintala_clipped.shp")
